# Explicamelo como si tuviera 5 años


![Architecture](images/architecture.png)

En este cuaderno vamos a recorrer el proceso de configurar un chatbot simple en LangChain.

A lo largo de este proceso, vamos a mostrar cómo LangSmith puede usarse para mejorar la experiencia del desarrollador en aplicaciones de IA.

Empecemos cargando nuestras variables de entorno desde el archivo .env.

In [20]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)
# Cargar las siguientes variables de entorno:
# LANGSMITH_TRACING=true
# LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
# LANGSMITH_PROJECT="eli5-bot"
# LANGSMITH_API_KEY="<redacted>"

# OPENAI_API_KEY="<redacted>"
# TAVILY_API_KEY="<redacted>"

True

## Setup

Configuremos una herramienta llamada Tavily para permitir que nuestro asistente busque en la web al responder.

In [21]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=1)

Diseñemos un prompt para RAG que usaremos a lo largo de todo el cuaderno.

In [22]:
prompt = """Sos un profesor y un experto en explicar temas complejos de una manera fácil de entender.
Tu trabajo es responder la pregunta dada de forma tal que incluso un niño de 5 años pueda comprenderla.
Se te ha proporcionado el contexto necesario para responder la pregunta.

Pregunta: {question} 

Contexto: {context}

Respuesta:"""
print("Prompt Template: ", prompt)

Prompt Template:  Sos un profesor y un experto en explicar temas complejos de una manera fácil de entender.
Tu trabajo es responder la pregunta dada de forma tal que incluso un niño de 5 años pueda comprenderla.
Se te ha proporcionado el contexto necesario para responder la pregunta.

Pregunta: {question} 

Contexto: {context}

Respuesta:


## Creando nuestra aplicación

In [23]:
from openai import OpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai

openai_client = wrap_openai(OpenAI())

@traceable
def search(question):
    web_docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in web_docs])
    return web_results
    
@traceable
def explain(question, context):
    formatted = prompt.format(question=question, context=context)
    
    completion = openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": formatted},
            {"role": "user", "content": question},
        ],
        model="gpt-4o-mini",
    )
    return completion.choices[0].message.content

@traceable
def eli5(question):
    context = search(question)
    answer = explain(question, context)
    return answer

## Testeando nuestra application

In [24]:
question = "Qué es la macroeconomía?"
print(eli5(question))

¡Claro! Imagina que la economía es como una gran fiesta con muchas personas. La macroeconomía es la parte que mira a toda la fiesta en lugar de solo a una persona o a un grupo pequeño.

Entonces, en vez de fijarnos en lo que le pasa a un niño con su juguete, la macroeconomía observa cosas como cuántos niños están en la fiesta, si hay suficiente comida para todos, y si están todos felices o no. Eso incluye cosas como si la gente tiene trabajo, si los precios de las cosas suben o bajan, y cómo los adultos (los gobiernos) tratan de que todo funcione mejor.

Así que, en resumen, la macroeconomía nos ayuda a entender cómo va la fiesta grande que es la economía de un país.
